# CS242 Machine Learning HW3
## Question 2. Perceptron Algorithm

In [1]:
import numpy as np

### Creating X

In [2]:
instance_no = 300
feature_no = 15
random_str = np.random.binomial(1, 0.5, instance_no*feature_no)
random_str[random_str == 0] = -1
X = random_str.reshape((instance_no,feature_no))

### Traning perceptron function 1 (until Y_predict is the same as Y)

In [3]:
def perceptron_training1(X,Y):
    no_epoch = 0
    no_error = 0
    w = np.zeros(feature_no)
    Y_predict = np.sign(np.dot(X,w))
    while np.array_equal(Y,Y_predict) == False:
        for i, j in enumerate(X):
            if (np.dot(X[i], w)*Y[i]) <= 0:
                w = w + X[i]*Y[i]
                no_error += 1
#                 print(i)
        Y_predict = np.sign(np.dot(X,w))
        no_epoch += 1 
    return no_error, no_epoch, w

## Question 2a
### Results for 2a:

In [4]:
Ya = X[:,0]

no_error_a, no_epoch_a, w_a = perceptron_training1(X,Ya)
print('Number of errors: ', no_error_a)
print('Number of epochs: ', no_epoch_a)

Number of errors:  10
Number of epochs:  1


## Question 2b
### Results for 2b:

In [5]:
Yb = np.zeros(instance_no)
for i in range(instance_no):
    if np.sum(X[i])>0:
        Yb[i] = 1
    if np.sum(X[i])<0:
        Yb[i] = -1

no_error_b, no_epoch_b, w_b = perceptron_training1(X,Yb)
print('Number of errors: ', no_error_b)
print('Number of epochs: ', no_epoch_b)     

Number of errors:  69
Number of epochs:  3


## Question 2c
### Creating Yc (with noise r), 500 instances X_test & Y_test

In [6]:
Y_train = np.zeros(instance_no)
for i in range(instance_no):
    r = np.random.uniform(low=-4.0, high=4.0)
    Y_train[i] = np.sign(r + np.sum(X[i,j] for j in range(11)))
    
test_instance_no = 500
test_random_str = np.random.binomial(1, 0.5, test_instance_no*feature_no)
test_random_str[test_random_str == 0] = -1
X_test = test_random_str.reshape((test_instance_no,feature_no))    
    
Y_test = np.zeros(test_instance_no)
for i in range(test_instance_no):
    r = np.random.uniform(low=-4.0, high=4.0)
    Y_test[i] = np.sign(r + np.sum(X_test[i,j] for j in range(11)))

### Perceptron training + saving all w vectors 

In [7]:
def perceptron_training2(X,Y,W):
    no_error = 0
    for i, j in enumerate(X):
        if (np.dot(X[i], W[i])*Y[i]) <= 0 and i<instance_no-1:
#             print(i)
            W[i+1] = W[i] + X[i]*Y[i]
            no_error += 1
    Y_predict = np.sign(np.dot(X,W[-1]))
    return no_error, W

### Shuffle X and W without losing corespondence

In [8]:
def shuffle(X,W):
    XW1 = np.hstack((X,W))
    np.random.shuffle(XW1)
    X_shuffled = XW1[:,0:15]
    W_shuffled = XW1[:,15:30]
    return X_shuffled, W_shuffled

In [9]:
W0 = np.zeros(np.shape(X))
no_error1, W1 = perceptron_training2(X,Y_train, W0)
X1, W1 = shuffle(X,W1)
no_error2, W2 = perceptron_training2(X1,Y_train, W1)
X2, W2 = shuffle(X1,W1)
no_error3, W3 = perceptron_training2(X2,Y_train, W2)
print(no_error1)
print(no_error2)
print(no_error3)

200
175
152


In [10]:
# W0 = np.zeros(np.shape(X))
# no_error1, W1 = perceptron_training2(X,Y_train, W0)
# no_error2, W2 = perceptron_training2(X,Y_train, W1)
# no_error3, W3 = perceptron_training2(X,Y_train, W2)
# print(no_error1)
# print(no_error2)
# print(no_error3)

### i) The last hypothesis predicts with $sign(w_{900} \cdot x)$, using the last weight vector from the run

In [11]:
Y_predict1 = np.sign(np.dot(X_test,W3[-1]))
np.sum(Y_test==Y_predict1)/500

0.42999999999999999

### ii) The average hypothesis computes $\bar{w} = \sum_{j=1}^{900} w_j /900$ and predicts with $sign(\bar{w}\cdot x)$

In [12]:
W_concat = np.vstack((W1,W2,W3))
W_bar = np.sum(W_concat, axis=0)/900
Y_predict2 = np.sign(np.dot(X_test,W_bar))
np.sum(Y_test==Y_predict2)/500

0.68000000000000005

### iii) The voted hypothesis votes all 900 weight vectors, predicting with $sign\left( \sum_{i=1}^{900} sign(w_i \cdot x)\right)$ (if the vote is tied, treat it as an incorrect prediction)

In [13]:
W_concat = np.vstack((W1,W2,W3))
vote = np.zeros(500)
for i in range(900):
    vote_temp = np.sign(np.dot(X_test,W_concat[i])) 
    vote += vote_temp
Y_predict3 = np.sign(vote)
np.sum(Y_test==Y_predict3)/500

0.66800000000000004

### iv) The last epoch average computes $\bar{w}_l = \sum_{j=601}^{900}w_j/300$ and predicts with $sign(\bar{w} \cdot x)$

In [14]:
W_lbar = np.sum(W3, axis=0)/300
Y_predict2 = np.sign(np.dot(X_test,W_bar))
np.sum(Y_test==Y_predict2)/500

0.68000000000000005

### v) The last epoch vote predicts with the vote $sign\left( \sum_{i=601}^{900} sign(w_i \cdot x)\right)$

In [15]:
vote = np.zeros(500)
for i in range(300):
    vote_temp = np.sign(np.dot(X_test,W3[i])) 
    vote += vote_temp
Y_predict3 = np.sign(vote)
np.sum(Y_test==Y_predict3)/500

0.61199999999999999